# FSDS Group Assessment (Group Safari)

### <span style="color:red">If you have run section 1 before, please start from section 2.

# 1. Data Collection and Cleaning

We will use 2 different datasets:
1. Airbnb data of London (10 Dec, 2022) downloading from [InsideAirbnb](http://insideairbnb.com/get-the-data)  
2. 2011 and 2021 Census data including:
* popchurn 11.csv
* MIG009EW_LTLA_OUT.csv
* MIG009EW_LTLA_IN.csv
* ethnic group 2011.csv
* ethnic group 2021.csv
* house price_median.xls
* house price_aver.xlsx
* Deprivation 2011.xls
* Deprivation 2021.csv


Note that all data in the Data subdirectory is ignored in the `.gitignore` file.

The file names that are used in this script are as follows.

|Data Type|File Name|df/gdf name|Gentrification Score df Name|Note|
|:---|:---|:---|:--|:--|
||`popchurn 11.csv`|`popch2011`|`2011migration`||
||`MIG009EW_LTLA_OUT.csv`|`moving2021`|`2021migration`||
||`MIG009EW_LTLA_IN.csv`||||
||`ethnic group 2011.csv`|`eg2011`|`nw_ratio11`||
||`ethnic group 2021.csv`|`eg2021`|`nw_ratio21`||
||`house price_median.xls`|`price_med`|`houseprice_change`||
||`house price_aver.xlsx`|`housing_df`|||
||`Deprivation 2011.xls`|`dpr2011`|`dpr2011%`||
||`Deprivation 2021.csv`|`dpr2021`|`dpr2021%`||
|`multipolygon`|`Boroughs.gpkg`|`boros`||`crs:27700`|
|`Airbnb listing`|`listings.csv.gz`|`ls`|`\`||
|`Airbnb listing -> geopandas`|`\`|`gls`|`\`|`crs:27700`|


## 1.1 Get Prepared

In [457]:
# Import packages
import os
from urllib.request import urlopen
from requests import get
from urllib.parse import urlparse
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [312]:
# Download data from remote location
def cache_data(src:str, dest:str) -> str:
    """Downloads and caches a remote file locally.
    
    The function sits between the 'read' step of a pandas or geopandas
    data frame and downloading the file from a remote location. The idea
    is that it will save it locally so that you don't need to remember to
    do so yourself. Subsequent re-reads of the file will return instantly
    rather than downloading the entire file for a second or n-th itme.
    
    Parameters
    ----------
    src : str
        The remote *source* for the file, any valid URL should work.
    dest : str
        The *destination* location to save the downloaded file.
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """
    url = urlparse(src)
    fn  = os.path.split(url.path)[-1]
    dfn = os.path.join(dest,fn)
    
    if not os.path.isfile(dfn):
        print(f"{dfn} not found, downloading!")
        path = os.path.split(dest)
        
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)  
        print("\tDone downloading...")
    else:
        print(f"Found {dfn} locally!")
        
    return dfn

Please save data files under directory: ***Data/*** which is in the same level as this ipynb file

In [313]:
#local_repo_dir = 'Documents/casa/fsds/group' # change this to your own directory under 'work'
# os.chdir('/home/jovyan/work/' + local_repo_dir)
padir = 'data/'

## 1.2 Gentrification Score

### 1.2.1 Load the Data of Each Gentrification Index
<span style="color:red">**For now, I am using local files(which are under folder "data"), but I'll adjust it later to download directly using url.** </span>  

(a) Population Churn (migration)

In [342]:
# 2011 data
popch2011 = pd.read_csv(padir+'popchurn 11.csv', skiprows=7, skip_blank_lines=True, usecols=[
    'local authority: district / unitary (prior to April 2015)',
    'mnemonic',
    'Whole household lived at same address one year ago', 
    'Wholly moving household: Total']).dropna(how='all').iloc[:33]

# 2021 data
popch2021_in_raw = pd.read_csv(padir + 'MIG009EW_LTLA_IN.csv', usecols=['Lower tier local authorities code', 'Household migration LTLA (inflow) (7 categories) code', 'Count'])
popch2021_out_raw = pd.read_csv(padir + 'MIG009EW_LTLA_OUT.csv', usecols=['Migrant LTLA one year ago code', 'Household migration LTLA (outflow) (3 categories) code', 'Count'])

popch2021_in = popch2021_in_raw.loc[popch2021_in_raw['Lower tier local authorities code'].astype(str).str.match(r'^E090000[0-9]{2}$|^E09000[1-3][0-3]$', na=False)]
popch2021_out = popch2021_out_raw.loc[popch2021_out_raw['Migrant LTLA one year ago code'].astype(str).str.match(r'^E090000[0-9]{2}$|^E09000[1-3][0-3]$', na=False)]

## establish the dataframe and select the data 2021 based on the definition
popch2021 = pd.DataFrame()
popch2021['samead_2021'] = popch2021_in.loc[popch2021_in['Household migration LTLA (inflow) (7 categories) code'] == 1].groupby('Lower tier local authorities code')['Count'].sum()
popch2021['movein_2021']= popch2021_in.loc[(popch2021_in['Household migration LTLA (inflow) (7 categories) code'] >= 2) & (popch2021_in['Household migration LTLA (inflow) (7 categories) code'] <= 5)].groupby('Lower tier local authorities code')['Count'].sum()
popch2021['moveout_2021'] = popch2021_out.loc[(popch2021_out['Household migration LTLA (outflow) (3 categories) code'] >= 1) & (popch2021_out['Household migration LTLA (outflow) (3 categories) code'] <= 2)].groupby('Migrant LTLA one year ago code')['Count'].sum()

(b) Ethnic Group

In [357]:
#data,2011 and 2021
eg2011 = pd.read_csv(padir+'ethnic group 2011.csv', skiprows=7, header=0, skip_blank_lines=True, usecols=[
    'mnemonic','All categories: Ethnic group','White'])
eg2021 = pd.read_csv(padir+'ethnic group 2021.csv', skiprows=6, header=0, skip_blank_lines=True, usecols=[
    'mnemonic','Total: All usual residents','White'])
# select data by borough code, 2011 and 2021
eg2011 = eg2011[eg2011['mnemonic'].astype(str).str.startswith('E09')]
eg2021 = eg2021[eg2021['mnemonic'].astype(str).str.startswith('E09')]
# set the borough code as index
eg2011.set_index('mnemonic', inplace=True)
eg2021.set_index('mnemonic', inplace=True)

(c) Housing Price

In [165]:
# median housing price,2011 and 2021
price_med_raw = pd.read_excel(padir+'house price_median.xls',sheet_name='1a',engine='xlrd',skiprows=5,header=0,usecols=[
    'Local authority code','Year ending Dec 2001','Year ending Dec 2021'])
price_med = price_med_raw.loc[price_med_raw['Local authority code'].astype(str).str.contains(r'^E09', regex=True)]
price_med.set_index('Local authority code', inplace=True)

# select data in 2011 and 2021
Housing_med_df = pd.DataFrame()
Housing_med_df ['median_2011'] =price_med.loc[:, ['Year ending Dec 2001']]
Housing_med_df['median_2021'] =price_med.loc[:, ['Year ending Dec 2021']]
Housing_med_df = Housing_med_df.groupby('Local authority code')[['median_2011', 'median_2021']].median()

In [170]:
# average housing price,2011 and 2021 
housing_price = "house price_aver.xlsx"
housing_df = pd.read_excel(os.path.join(padir,housing_price),sheet_name=2,skiprows=1, header=0,index_col=0)

# set the index to datetime data
housing_df.index = pd.to_datetime(housing_df.index, format='%Y%m%d')
# set the column and index name
housing_df.columns.name = 'London_borough'
housing_df.index.name = 'year'
# check the index(year) type
print(housing_df.index.dtype)
# select the london borough data
London_housing_df = housing_df.filter(regex='^E09', axis=1)
# change the column and index location 
London_housing_df = London_housing_df.transpose()
# check the data
London_housing_df.head(3) 

# select the data of 2011 and 2021
housing_ave_df = pd.DataFrame()
housing_ave_df ['average_2011'] =London_housing_df.loc[:, ['2011-12-01']]
housing_ave_df ['average_2021'] =London_housing_df.loc[:, ['2012-12-01']]
housing_ave_df.head(10) 

datetime64[ns]


,average_2011,average_2021
London_borough,,
E09000001,496290.8438,491528.7635
E09000002,162631.3061,167813.0248
E09000003,346652.9974,361370.6666
E09000004,199523.2257,206956.1398
E09000005,301759.5506,322170.5341
E09000006,278849.1468,288681.6021
E09000007,547656.5284,620350.6021
E09000008,217808.9162,226742.0544
E09000009,303094.5273,327287.6027


(d) Deprivation

In [36]:
# data 2011
dpr2011_raw = pd.read_excel(padir+'deprivation 2011.xls',sheet_name='QS119EW_Percentages',engine='xlrd',skiprows=10,header=0, usecols=[
    'Area code','Household is not deprived in any dimension'])
dpr2011 = dpr2011_raw.loc[dpr2011_raw['Area code'].astype(str).str.contains(r'^E090000[0-2][0-9]$|^E090003[0-3]$|^E090000[1-9][0-9]$|^E09000[1-3][0-3]$'
, regex=True)]

# data 2021
dpr2021_raw = pd.read_csv(padir+'deprivation 2021.csv')
dpr2021 = dpr2021_raw[dpr2021_raw['Upper tier local authorities Code'].astype(str).str.contains(
    r'^E090000[0-2][0-9]$|^E090003[0-3]$|^E090000[1-9][0-9]$|^E09000[1-3][0-3]$', regex=True)]

### 1.2.2 Calculate Each Gentrification index
G = 1/2c - 1/4e + 1/8h - 1/8d + 0.25  
    G: gentrification score  
    c: population churn at household level - the ratio of the households that have changed  
    e: ethnic group - the change of the proportion of non-white residents  
    h: housing price - relative change in median house price compared with acerage price  
    d: deprivation - relative change in the proportion of households with deprivation dimensions

**(0)G: establish dataframe for gentrification score**

In [498]:
gtr = pd.DataFrame()

**(a) c: Population Churn (Household Level)**  
* give a equation

In [499]:
## 2011 migrantion (household level)
popch2011['2011migration'] = (
    (popch2011['Wholly moving household: Total'] /
    (popch2011['Wholly moving household: Total'] + popch2011['Whole household lived at same address one year ago'])))

gtr['borough'] = popch2011['local authority: district / unitary (prior to April 2015)']
gtr['borough code'] = popch2011['mnemonic'].astype(str)
gtr['2011migration'] = popch2011['2011migration']
gtr.set_index('borough code', inplace=True)

In [500]:
## 2021 migrantion (household level)
popch2021['2021migration'] = (popch2021['movein_2021'] + 
                              popch2021['moveout_2021'])/ (popch2021['samead_2021'] + 
 popch2021['movein_2021'] + popch2021['moveout_2021'])
# Merge the result of population churn into gtr based on 'borough code' and 'code'
gtr ['2021migration']= popch2021['2021migration']
# add 'popchurn' column: 
gtr['popchurn'] = gtr['2021migration'] - gtr['2011migration']

**(b) e: Non-white Ethnic Group Proportion Change**  
* give a equation

In [501]:
# Calculate the no-white ratio for 2011
eg2011['nw_ratio11'] = (1 - eg2011['White'] / eg2011['All categories: Ethnic group'])
# Calculate the no-white ratio for 2021
eg2021['nw_ratio21'] = (1 - eg2021['White'] / eg2021['Total: All usual residents'])

In [502]:
# links the no-white ratio to gtr dataframe
gtr ['nw_ratio11'] = eg2011['nw_ratio11']
gtr ['nw_ratio21']= eg2021['nw_ratio21'] 
# add 'ethnic group change' column
gtr['ethgr'] = (gtr['nw_ratio21'] - gtr['nw_ratio11'])/ eg2011['nw_ratio11']

**(c) h: Housing Price Change (Median/Average)**  
* give a equation

In [503]:
# link the median data and average data
total_housing_df = pd.merge(housing_ave_df,Housing_med_df, left_index=True, right_index=True)
# calculate the change of housing price
total_housing_df['Housing_Compare_2011'] = total_housing_df['median_2011']/total_housing_df['average_2011']
total_housing_df['Housing_Compare_2021'] = total_housing_df['median_2021']/total_housing_df['average_2021']
total_housing_df['houseprice_change'] = (total_housing_df['Housing_Compare_2021']-total_housing_df['Housing_Compare_2011']) / total_housing_df['Housing_Compare_2011']

# link the housing data set to grf
gtr = pd.merge(gtr, total_housing_df[['Housing_Compare_2011','Housing_Compare_2021','houseprice_change']], left_index=True, right_index=True)

**(d) d: Deprivation Proportion Change**  
* give a equation

In [504]:
# select the 2021 data by the definition of index
dpr2021_nodpr = dpr2021[dpr2021['Household deprivation (6 categories) Code'] == 1]
dpr2021_all = dpr2021[(dpr2021['Household deprivation (6 categories) Code'] >= 1) & (dpr2021['Household deprivation (6 categories) Code'] <= 5)]
sum = dpr2021_all.groupby('Upper tier local authorities Code')['Observation'].sum()

ratios = 1- (dpr2021_nodpr.groupby('Upper tier local authorities Code')['Observation'].sum() / sum)*100 

# Create a new DataFrame by merging 'dpr2011' and 'ratios'
Dep_result_df = pd.merge(dpr2011, ratios, left_on='Area code', right_index=True, how='left')
Dep_result_df = Dep_result_df.rename(columns={'Observation': 'dpr2021%',
                                      'Household is not deprived in any dimension': 'dpr2011%'})
Dep_result_df.set_index('Area code', inplace=True)

In [505]:
# links the deprivation index to gtr dataset
gtr['dpr2011%']= Dep_result_df['dpr2011%']
gtr['dpr2021%']= Dep_result_df['dpr2021%']
gtr['dpr'] = (gtr['dpr2021%'] - gtr['dpr2011%'])/gtr['dpr2011%']

In [506]:
# check the final result of gtr dataframe
gtr

,borough,2011migration,2021migration,popchurn,nw_ratio11,nw_ratio21,ethgr,Housing_Compare_2011,Housing_Compare_2021,houseprice_change,dpr2011%,dpr2021%,dpr
E09000002,Barking and Dagenham,0.113141,0.110965,-0.002176,0.417980,0.550991,0.318225,0.540310,1.988821,2.680891,28.2,-36.591675,-2.297577
E09000003,Barnet,0.135282,0.134497,-0.000785,0.359116,0.422716,0.177101,0.533675,1.660345,2.111155,43.2,-48.573264,-2.124381
E09000004,Bexley,0.085597,0.092283,0.006686,0.183877,0.281209,0.529329,0.601352,1.903785,2.165840,41.5,-47.518872,-2.145033
E09000005,Brent,0.140259,0.151894,0.011635,0.638321,0.653638,0.023996,0.525770,1.580685,2.006416,30.9,-38.941996,-2.260259
E09000006,Bromley,0.109365,0.106850,-0.002514,0.158705,0.235449,0.483565,0.568408,1.697372,1.986187,48.5,-53.642302,-2.106027
E09000007,Camden,0.193798,0.210687,0.016888,0.337890,0.404843,0.198149,0.455988,1.302691,1.856851,37.9,-46.523151,-2.227524
E09000001,City of London,0.270857,0.322132,0.051275,0.214102,0.306267,0.430477,0.478550,1.622489,2.390427,45,-58.768010,-2.305956
E09000008,Croydon,0.115195,0.119348,0.004153,0.449716,0.516321,0.148104,0.573898,1.758606,2.064321,41,-46.994717,-2.146213
E09000009,Ealing,0.137726,0.140970,0.003244,0.510951,0.568357,0.112351,0.544385,1.588817,1.918555,37.4,-45.034656,-2.204135
E09000010,Enfield,0.120092,0.109127,-0.010965,0.390987,0.479116,0.225402,0.532801,1.726798,2.240980,36.1,-41.187397,-2.140925


### 1.2.3 Calculate Gentrification Score
G = 1/2c - 1/4e + 1/8h - 1/8d + 0.25  
    G: gentrification score  

In [507]:
gtr['score'] = (1/2*gtr['popchurn'] - 1/4*gtr['ethgr'] 
                + 1/8*gtr['houseprice_change'] -1/8*gtr['dpr']) + 0.25

In [508]:
# reset the index and check the result of gtr dataframe and
gtr.reset_index(inplace=True)
gtr

,index,borough,2011migration,2021migration,popchurn,nw_ratio11,nw_ratio21,ethgr,Housing_Compare_2011,Housing_Compare_2021,houseprice_change,dpr2011%,dpr2021%,dpr,score
0,E09000002,Barking and Dagenham,0.113141,0.110965,-0.002176,0.417980,0.550991,0.318225,0.540310,1.988821,2.680891,28.2,-36.591675,-2.297577,0.791664
1,E09000003,Barnet,0.135282,0.134497,-0.000785,0.359116,0.422716,0.177101,0.533675,1.660345,2.111155,43.2,-48.573264,-2.124381,0.734774
2,E09000004,Bexley,0.085597,0.092283,0.006686,0.183877,0.281209,0.529329,0.601352,1.903785,2.165840,41.5,-47.518872,-2.145033,0.65987
3,E09000005,Brent,0.140259,0.151894,0.011635,0.638321,0.653638,0.023996,0.525770,1.580685,2.006416,30.9,-38.941996,-2.260259,0.783153
4,E09000006,Bromley,0.109365,0.106850,-0.002514,0.158705,0.235449,0.483565,0.568408,1.697372,1.986187,48.5,-53.642302,-2.106027,0.639378
5,E09000007,Camden,0.193798,0.210687,0.016888,0.337890,0.404843,0.198149,0.455988,1.302691,1.856851,37.9,-46.523151,-2.227524,0.719454
6,E09000001,City of London,0.270857,0.322132,0.051275,0.214102,0.306267,0.430477,0.478550,1.622489,2.390427,45,-58.768010,-2.305956,0.755066
7,E09000008,Croydon,0.115195,0.119348,0.004153,0.449716,0.516321,0.148104,0.573898,1.758606,2.064321,41,-46.994717,-2.146213,0.741367
8,E09000009,Ealing,0.137726,0.140970,0.003244,0.510951,0.568357,0.112351,0.544385,1.588817,1.918555,37.4,-45.034656,-2.204135,0.738871
9,E09000010,Enfield,0.120092,0.109127,-0.010965,0.390987,0.479116,0.225402,0.532801,1.726798,2.240980,36.1,-41.187397,-2.140925,0.735905


### 1.2.4 Save the file of gentrification score to the local

In [510]:
# files saved under data/
save_path = 'gentrification_score.csv' 
full_path = os.path.join(padir, save_path)
gtr.to_csv(full_path, index=False)

### 1.3 Airbnb data

In [511]:
# boroughs
boros = gpd.read_file(cache_data('https://github.com/jreades/fsds/blob/master/data/src/Boroughs.gpkg?raw=true', padir+'Boroughs.gpkg') )
boros.crs
print(boros.head(10))

# listings
url = 'http://data.insideairbnb.com/united-kingdom/england/london/2022-12-10/data/listings.csv.gz'
ls = pd.read_csv(cache_data(url, padir),compression='gzip',usecols=['id','latitude','longitude','price','bedrooms','minimum_nights'])
print(ls.head(10))

Found data/Boroughs.gpkg/Boroughs.gpkg locally!
                   NAME   GSS_CODE   HECTARES  NONLD_AREA ONS_INNER  \
0  Kingston upon Thames  E09000021   3726.117       0.000         F   
1               Croydon  E09000008   8649.441       0.000         F   
2               Bromley  E09000006  15013.487       0.000         F   
3              Hounslow  E09000018   5658.541      60.755         F   
4                Ealing  E09000009   5554.428       0.000         F   
5              Havering  E09000016  11445.735     210.763         F   
6            Hillingdon  E09000017  11570.063       0.000         F   
7                Harrow  E09000015   5046.330       0.000         F   
8                 Brent  E09000005   4323.270       0.000         F   
9                Barnet  E09000003   8674.837       0.000         F   

                                            geometry  
0  MULTIPOLYGON (((516401.600 160201.800, 516407....  
1  MULTIPOLYGON (((535009.200 159504.700, 535005....  
2  MU

In [512]:
# ls to Gegpd
gls = gpd.GeoDataFrame(ls, 
      geometry=gpd.points_from_xy(ls.longitude, ls.latitude, crs='epsg:27700'))
gls['price'] = gls['price'].str.replace('$','', regex=False).str.replace(',','', regex=False).astype('float')
print(gls.head(10))

       id   latitude  longitude  bedrooms  price  minimum_nights  \
0   13913  51.568610  -0.112700       1.0   79.0               1   
1   15400  51.487800  -0.168130       1.0   75.0              10   
2  172811  51.547100  -0.179810       2.0  229.0              21   
3  173082  51.538254  -0.044086       1.0  132.0               2   
4   42010  51.585900  -0.164340       1.0   65.0               4   
5   17402  51.521950  -0.140940       3.0  425.0               4   
6   42692  51.508220  -0.215620       1.0  125.0               1   
7   43129  51.481640  -0.210820       1.0   60.0               3   
8   43202  51.530310  -0.217130       1.0  141.0               4   
9   43742  51.551430  -0.094930       1.0   78.0               1   

                geometry  
0  POINT (-0.113 51.569)  
1  POINT (-0.168 51.488)  
2  POINT (-0.180 51.547)  
3  POINT (-0.044 51.538)  
4  POINT (-0.164 51.586)  
5  POINT (-0.141 51.522)  
6  POINT (-0.216 51.508)  
7  POINT (-0.211 51.482)  
8  POINT

In [294]:
# add gentrification score to borough dataframe
merged_df = pd.merge(boros, gtr[['borough code', 'score']], left_on='GSS_CODE', right_on='borough code', how='left')
merged_df = merged_df.drop('borough code', axis=1)

### <span style="color:red">If you have run section 1 before, please start from section 2.

# 2. Analysis

## 2.1 load the data 

### 2.1.1 Gentrification Score

In [513]:
file_name = 'gentrification_score.csv' 
gentrification_path = os.path.join(padir, file_name)
gentrification_score = pd.read_csv(gentrification_path)

### 2.1.2 Airbnb data

In [516]:
# set the borough code as index
gentrification_score.set_index('index',inplace=True)
# check the dataset
gentrification_score

,borough,2011migration,2021migration,popchurn,nw_ratio11,nw_ratio21,ethgr,Housing_Compare_2011,Housing_Compare_2021,houseprice_change,dpr2011%,dpr2021%,dpr,score
index,,,,,,,,,,,,,,
E09000002,Barking and Dagenham,0.113141,0.110965,-0.002176,0.417980,0.550991,0.318225,0.540310,1.988821,2.680891,28.2,-36.591675,-2.297577,0.791664
E09000003,Barnet,0.135282,0.134497,-0.000785,0.359116,0.422716,0.177101,0.533675,1.660345,2.111155,43.2,-48.573264,-2.124381,0.734774
E09000004,Bexley,0.085597,0.092283,0.006686,0.183877,0.281209,0.529329,0.601352,1.903785,2.165840,41.5,-47.518872,-2.145033,0.659870
E09000005,Brent,0.140259,0.151894,0.011635,0.638321,0.653638,0.023996,0.525770,1.580685,2.006416,30.9,-38.941996,-2.260259,0.783153
E09000006,Bromley,0.109365,0.106850,-0.002514,0.158705,0.235449,0.483565,0.568408,1.697372,1.986187,48.5,-53.642302,-2.106027,0.639378
E09000007,Camden,0.193798,0.210687,0.016888,0.337890,0.404843,0.198149,0.455988,1.302691,1.856851,37.9,-46.523151,-2.227524,0.719454
E09000001,City of London,0.270857,0.322132,0.051275,0.214102,0.306267,0.430477,0.478550,1.622489,2.390427,45.0,-58.768010,-2.305956,0.755066
E09000008,Croydon,0.115195,0.119348,0.004153,0.449716,0.516321,0.148104,0.573898,1.758606,2.064321,41.0,-46.994717,-2.146213,0.741367
E09000009,Ealing,0.137726,0.140970,0.003244,0.510951,0.568357,0.112351,0.544385,1.588817,1.918555,37.4,-45.034656,-2.204135,0.738871
